In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: user 271 ms, sys: 205 ms, total: 476 ms
Wall time: 3.25 s


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2_fromtext.yaml"
conversation = None

In [4]:
print(image_path)

images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg


In [5]:
images = sorted(utils.load_images(image_dir))[:2]

In [6]:
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 100000, batch_size=1, temperature=0.4)

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `

In [7]:
outputs = model(images, save=True, debug=False)

Model: qwen_model | Batch Size: 1, Max Tokens: 100000, Temperature: 0.4
Extracting Text from Images


Processing Batches: 100%|██████████| 2/2 [00:56<00:00, 28.01s/batch]


Converting extracted text into Text Blocks
Organising text into JSON blocks


Processing Families in Dicotyledones: 100%|██████████| 9/9 [01:07<00:00,  7.49s/family]
